In [1]:
import numpy as np

### Runge-Kutta 4: dt step

\begin{equation*}
\frac{dx}{dt}=f(x)\Rightarrow x(t+dt) = \mathrm{rk4\_step}(x(t), f, dt)
\end{equation*}

In [2]:
def rk4_step(x, f, dt):
    a = f(x)
    b = f(x+0.5*dt*a)
    c = f(x+0.5*dt*b)
    d = f(x+dt*c)
    return x+dt*(a+2*b+2*c+d)/6.0

### Planetary motion: f and E

\begin{equation*}
H(r_x, r_y, p_x, p_y) = \frac{p_x^2+p_y^2}{2}-\frac{1}{\sqrt{r_x^2+r_y^2}}
\end{equation*}
\begin{equation*}
\dot{r_x} = \frac{\partial H}{\partial p_x} = p_x
\end{equation*}
\begin{equation*}
\dot{r_y} = \frac{\partial H}{\partial p_y} = p_y
\end{equation*}
\begin{equation*}
\dot{p_x} = -\frac{\partial H}{\partial r_x} = -\frac{r_x}{(r_x^2+r_y^2)^{\frac{3}{2}}}
\end{equation*}
\begin{equation*}
\dot{p_y} = -\frac{\partial H}{\partial r_y} =  -\frac{r_y}{(r_x^2+r_y^2)^{\frac{3}{2}}}
\end{equation*}

\begin{equation*}
\Rightarrow x = (r_x, r_y, p_x, p_y)
\end{equation*}

\begin{equation*}
\Rightarrow f = \Bigg(p_x, p_y, -\frac{r_x}{(r_x^2+r_y^2)^{\frac{3}{2}}},  -\frac{r_y}{(r_x^2+r_y^2)^{\frac{3}{2}}}\Bigg)
\end{equation*}

In [3]:
def bmg(x):
    tmp = -1.0/(x[0]**2+x[1]**2)**(1.5)
    return np.array([x[2], x[3], x[0]*tmp, x[1]*tmp], dtype=np.double)

def E(x):
    return 0.5*(x[2]**2+x[3]**2)-1.0/np.sqrt(x[0]**2+x[1]**2)

### Initial values

In [4]:
dt   = 0.01
t0   = 0
tmax = 100
x0 = np.array([0, 1.0/np.sqrt(10), 2.4, 0], dtype=np.double)
E0 = E(x0)

In [5]:
E0

-0.28227766016837919

### Time evolution

In [6]:
t = t0
x = x0
solution = [[t, x[0], x[1], x[2], x[3], E(x)-E0]]
while True:
    x = rk4_step(x, bmg, dt)
    t = t+dt
    solution.append([t, x[0], x[1], x[2], x[3], E(x)-E0])
    if t>=tmax:
        break

### Saving solution

In [7]:
f = open("clbjw0_bead1_sol.txt", "w")
for line in solution:
    f.write(" ".join(map(str, line))+"\n")
f.close()

### Plots

In [8]:
solution = np.array(solution)

In [9]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode()

# Hack for latex
from IPython.display import display, HTML
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))

In [10]:
fig = go.Figure(
        data   = [go.Scatter(x=solution[:, 1], y=solution[:, 2])], 
        layout = go.Layout(title='r$r_y \mathrm{vs.} r_x$', xaxis=dict(title="$r_x$"), yaxis=dict(title="$r_y$"))
    )

iplot(fig)

In [13]:
layout = go.Layout(
    title = "Energy as function of time",
    yaxis=dict(title = "E(t)-E(0)", exponentformat='power'),
    xaxis=dict(title = "t")
)
fig = go.Figure(data=[go.Scatter(x=solution[:, 0], y=solution[:, 5])], layout=layout)
iplot(fig)